In [1]:
'''
Author: Anmol Kachroo

'''

'\nAuthor: Anmol Kachroo\n\n'

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [114]:
import nltk
from nltk.corpus import twitter_samples 
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [115]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [116]:
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [119]:
#clean our tweets,remove unwanted words and char.
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
      if(word not in stopwords_english and word not in string.punctuation):
        stem_word = stemmer.stem(word)
        tweets_clean.append(stem_word)
    return tweets_clean

In [120]:
#check freq of words with pred. value
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
              freqs[pair] += 1
            else:
              freqs[pair] = 1
    return freqs

In [122]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')


In [123]:
#take data
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [124]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [125]:
Final_data = all_positive_tweets+all_negative_tweets
data =np.append(np.ones((len(all_positive_tweets), 1)), np.zeros((len(all_negative_tweets), 1)), axis=0)
train_x,test_x,train_y,test_y = train_test_split(Final_data,data,test_size=0.25,random_state= 53)

In [126]:
#our word dataset with freq
freqs = build_freqs(train_x , train_y)
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 10867


In [127]:
def extract_features(tweet, freqs): 
    word_l = process_tweet(tweet)
    x = np.zeros((1, 2)) 
    for word in word_l:
        if((word,1) in freqs):
          x[0,0]+=freqs[word,1]
        if((word,0) in freqs):
          x[0,1]+=freqs[word,0]
    
    assert(x.shape == (1, 2))
    return x[0]

In [128]:
#pred function
def predict_tweet(tweet):
    with tf.Session() as sess:
      saver.restore(sess,save_path='TSession')
      data_i=[]
      for t in tweet:
        data_i.append(extract_features(t,freqs))
      data_i=np.asarray(data_i)
      return sess.run(tf.nn.sigmoid(tf.add(tf.matmul(a=data_i,b=W,transpose_b=True),bias)))
    print("--Fail--")
    return

In [129]:
bias=tf.Variable(np.random.randn(1),name="Bias")
W=tf.Variable(np.random.randn(1,2),name="Weight")

In [130]:
data=[]
for t in train_x:
  data.append(extract_features(t,freqs))
data=np.asarray(data)

In [131]:
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(np.asarray(data), W,transpose_b=True), bias)) 
ta=np.asarray(train_y)
Total_cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_hat, labels = ta) 
print(Total_cost)

Tensor("logistic_loss:0", shape=(7500, 1), dtype=float64)


In [132]:
# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.00001 ,name="GradientDescent").minimize(Total_cost) 
# Global Variables Initializer 
init = tf.global_variables_initializer()

In [134]:
saver = tf.train.Saver()
with tf.Session() as sess:
  
  sess.run(init)
  print("Bias",sess.run(bias))
  print("Weight",sess.run(W))
  for epoch in range(1000):
    sess.run(optimizer)
    preds=sess.run(Y_hat)
    acc=((preds==ta).sum())/len(train_y)
    Accuracy=[]
    repoch=False
    if repoch:
      Accuracy.append(acc)
    if epoch % 1000 == 0:
      print("Accuracy",acc)
    saved_path = saver.save(sess, 'TSession')

preds=predict_tweet(test_x)
print(preds,len(test_y))

def accuracy(x,y):
  return ((x==y).sum())/len(y)

print(accuracy(preds,test_y))

Bias [0.77305999]
Weight [[0.78032969 0.86398166]]
Accuracy 0.49106666666666665
INFO:tensorflow:Restoring parameters from TSession
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]] 2500
0.5132
